# Obtaining a RCCSDT Code using Sympy and Tchau-Spin

$\bullet$ Import Sympy packages

In [1]:
from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator)
from sympy import (
    symbols, Rational, latex, Dummy
)

# For Sympy (simplification of equations)
pretty_dummies_dict = {
    'above': 'defgh',
    'below': 'lmnow',
    'general': 'pqrstu'
}

$\bullet$ Define symbols for Sympy

In [2]:
i = symbols('i', below_fermi=True, cls=Dummy)
a = symbols('a', above_fermi=True, cls=Dummy)
j = symbols('j', below_fermi=True, cls=Dummy)
b = symbols('b', above_fermi=True, cls=Dummy)
p, q, r, s = symbols('p,q,r,s', cls=Dummy)

## Build the $\Phi$-normal ordered Hamiltonian

In [3]:
fock = AntiSymmetricTensor('f', (p,), (q,))
pr = NO(Fd(p)*F(q))
V = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))
H = fock*pr + Rational(1,4)*V*pqsr
H

AntiSymmetricTensor(f, (_p,), (_q,))*NO(CreateFermion(_p)*AnnihilateFermion(_q)) - AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s))/4

## Build Cluster Operator

In [4]:
def get_T():
    i, j, k = symbols('i,j,k', below_fermi=True, cls=Dummy)
    a, b, c = symbols('a,b,c', above_fermi=True, cls=Dummy)
    t1 = AntiSymmetricTensor('t', (a,), (i,))*NO(Fd(a)*F(i))
    t2 = Rational(1,4)*AntiSymmetricTensor('t', (a,b), (i,j))*NO(Fd(a)*Fd(b)*F(j)*F(i))
    t3 = Rational(1,36)*AntiSymmetricTensor('t', (a,b,c), (i,j,k))*NO(Fd(a)*Fd(b)*Fd(c)*F(k)*F(j)*F(i))
    return t1 + t2 + t3

get_T()

AntiSymmetricTensor(t, (_a,), (_i,))*NO(CreateFermion(_a)*AnnihilateFermion(_i)) - AntiSymmetricTensor(t, (_a, _b), (_i, _j))*NO(CreateFermion(_a)*CreateFermion(_b)*AnnihilateFermion(_i)*AnnihilateFermion(_j))/4 - AntiSymmetricTensor(t, (_a, _b, _c), (_i, _j, _k))*NO(CreateFermion(_a)*CreateFermion(_b)*CreateFermion(_c)*AnnihilateFermion(_i)*AnnihilateFermion(_j)*AnnihilateFermion(_k))/36

## Perform the Hausdorff expansion

<center> $\bar{H} = e^{-\hat{T}}\hat{H}_Ne^{\hat{T}}$ <br/>
    
 <p>&nbsp;</p>
    
<center>$\bar{H} = \hat{H}_N + [\hat{H}_N, \hat{T}] + \frac{1}{2}[[\hat{H}_N, \hat{T}], \hat{T}] + \frac{1}{3!}[[[\hat{H}_N, \hat{T}], \hat{T}], \hat{T}] + \frac{1}{4!}[[[[\hat{H}_N, \hat{T}], \hat{T}], \hat{T}], \hat{T}]$

In [5]:
C = Commutator
T = get_T()
print("commutator 1...")
comm1 = wicks(C(H, T))
comm1 = evaluate_deltas(comm1)
comm1 = substitute_dummies(comm1)

commutator 1...


In [6]:
T = get_T()
print("commutator 2...")
comm2 = wicks(C(comm1, T))
comm2 = evaluate_deltas(comm2)
comm2 = substitute_dummies(comm2)

commutator 2...


In [7]:
T = get_T()
print("commutator 3...")
comm3 = wicks(C(comm2, T))
comm3 = evaluate_deltas(comm3)
comm3 = substitute_dummies(comm3)

commutator 3...


In [8]:
T = get_T()
print("commutator 4...")
comm4 = wicks(C(comm3, T))
comm4 = evaluate_deltas(comm4)
comm4 = substitute_dummies(comm4)

commutator 4...


## Construct the Similarity Transformed Hamiltonian

In [9]:
eq = H + comm1 + comm2/2 + comm3/6 + comm4/24
eq = eq.expand()
eq = evaluate_deltas(eq)
eq = substitute_dummies(eq, new_indices=True,
        pretty_indices=pretty_dummies_dict)

## Get energy expression

<center>$E = \langle | \bar{H} e^{\hat{T}}| \rangle_C$

In [10]:
i, j, k, l = symbols('i,j,k,l', below_fermi=True)
a, b, c, d = symbols('a,b,c,d', above_fermi=True)
energy = wicks(eq, simplify_dummies=True,
        keep_only_fully_contracted=True)
energy

AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (_d,), (_l,)) - AntiSymmetricTensor(t, (_d,), (_m,))*AntiSymmetricTensor(t, (_e,), (_l,))*AntiSymmetricTensor(v, (_l, _m), (_d, _e))/2 + AntiSymmetricTensor(t, (_d, _e), (_l, _m))*AntiSymmetricTensor(v, (_l, _m), (_d, _e))/4

# Get $T_1$ amplitude equation

<center>$E = \langle \Phi_{i}^{a}| \bar{H} e^{\hat{T}}| \rangle_C$

<p>&nbsp;</p>

<center> $E = \langle |i^\dagger a \bar{H} e^{\hat{T}}| \rangle_C$

In [11]:
eqT1 = wicks(NO(Fd(i)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
eqT1

-AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(t, (a,), (_l,)) + AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (a, _d), (i, _l)) - AntiSymmetricTensor(f, (_l,), (i,))*AntiSymmetricTensor(t, (a,), (_l,)) + AntiSymmetricTensor(f, (a,), (_d,))*AntiSymmetricTensor(t, (_d,), (i,)) + AntiSymmetricTensor(f, (a,), (i,)) - AntiSymmetricTensor(t, (_d,), (_l,))*AntiSymmetricTensor(t, (_e,), (i,))*AntiSymmetricTensor(t, (a,), (_m,))*AntiSymmetricTensor(v, (_l, _m), (_d, _e)) - AntiSymmetricTensor(t, (_d,), (_l,))*AntiSymmetricTensor(t, (_e,), (i,))*AntiSymmetricTensor(v, (a, _l), (_d, _e)) + AntiSymmetricTensor(t, (_d,), (_l,))*AntiSymmetricTensor(t, (a,), (_m,))*AntiSymmetricTensor(v, (_l, _m), (i, _d)) + AntiSymmetricTensor(t, (_d,), (_l,))*AntiSymmetricTensor(t, (a, _e), (i, _m))*AntiSymmetricTensor(v, (_l, _m), (_d, _e)) + AntiSymmetricTensor(t, (_d,), (_l,))*AntiSymmetricTensor(v, (a, _l), (i, _d)) - AntiSymmetricTensor(t, (_d,), (i

## Get $T_2$ amplitude equation

<center>$E = \langle \Phi_{ij}^{ab}| \bar{H} e^{\hat{T}}| \rangle_C$

<p>&nbsp;</p>

<center> $E = \langle |i^\dagger j^\dagger b a \bar{H} e^{\hat{T}}| \rangle_C$

In [12]:
eqT2 = wicks(NO(Fd(i)*Fd(j)*F(b)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
eqT2

AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _l)) - AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (_d,), (j,))*AntiSymmetricTensor(t, (a, b), (i, _l)) + AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (a,), (_l,))*AntiSymmetricTensor(t, (b, _d), (i, j)) - AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (b,), (_l,))*AntiSymmetricTensor(t, (a, _d), (i, j)) + AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (a, b, _d), (i, j, _l)) + AntiSymmetricTensor(f, (_l,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _l)) - AntiSymmetricTensor(f, (_l,), (j,))*AntiSymmetricTensor(t, (a, b), (i, _l)) - AntiSymmetricTensor(f, (a,), (_d,))*AntiSymmetricTensor(t, (b, _d), (i, j)) + AntiSymmetricTensor(f, (b,), (_d,))*AntiSymmetricTensor(t, (a, _d), (i, j)) + AntiSymmetricTensor(t, (_d,), (_l,))*AntiSymmetricTensor(t, (_e,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _m))*AntiSymmetricTensor(v, (_l, _

## Get $T_3$ amplitude equation

<center>$E = \langle \Phi_{ijk}^{abc}| \bar{H} e^{\hat{T}}| \rangle_C$

<p>&nbsp;</p>

<center> $E = \langle |i^\dagger j^\dagger k^\dagger c b a \bar{H} e^{\hat{T}}| \rangle_C$

In [13]:
eqT3 = wicks(NO(Fd(i)*Fd(j)*Fd(k)*F(c)*F(b)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
eqT3

-AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(t, (a, b, c), (j, k, _l)) + AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (_d,), (j,))*AntiSymmetricTensor(t, (a, b, c), (i, k, _l)) - AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (_d,), (k,))*AntiSymmetricTensor(t, (a, b, c), (i, j, _l)) - AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (a,), (_l,))*AntiSymmetricTensor(t, (b, c, _d), (i, j, k)) + AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (b,), (_l,))*AntiSymmetricTensor(t, (a, c, _d), (i, j, k)) - AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (c,), (_l,))*AntiSymmetricTensor(t, (a, b, _d), (i, j, k)) + AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (a, _d), (i, j))*AntiSymmetricTensor(t, (b, c), (k, _l)) - AntiSymmetricTensor(f, (_l,), (_d,))*AntiSymmetricTensor(t, (a, _d), (i, k))*AntiSymmetricTensor(t, (b, c), (j, _l)) + AntiSymmetricTensor(f, (_l,), (_d,)

# Using Tchau-Spin

In [14]:
from tchau_spin import *
# For restricted CC
Tensor.rhf = True

$\bullet$ Define indexes

In [15]:
i,j,k,a,b,c = Index.new('ijkabc', 'abaaba', 'hhhppp')
# 'xxxx' indicates that those indexes have any spin
l,m,d,e = Index.new('lmde', 'xxxx', 'hhpp')
index_key = {'i':i,
             'j':j,
             'a':a,
             'b':b,
             'k':k,
             'l':l,
             'c':c,
             'd':d,
             'l':l,
             'm':m,
             'e':e
}

## Processing Energy expression

In [16]:
E = eqfromlatex(latex(energy), index_key)
platex(E)

<IPython.core.display.Math object>

$\bullet$ Simplify Equation

In [17]:
E = E.simplify()
E = E.adapt_space()
platex(E)

<IPython.core.display.Math object>

$\bullet$ Process equation. It is necessary to identify the external indexes (in this case none). Also it is optional to provide names. Let's do it without these options first

In [18]:
pE = process_eq(eq=E)
pE.write_einsums_out(print_out=True)

name += 2.0*np.einsum('ld, ld -> ', f_OV, T_OV, optimize = 'optimal')
name -= np.einsum('md, le, lmde -> ', T_OV, T_OV, V_OOVV, optimize = 'optimal')
name += 2.0*np.einsum('md, le, mlde -> ', T_OV, T_OV, V_OOVV, optimize = 'optimal')
name -= np.einsum('mlde, lmde -> ', T_OoVv, V_OOVV, optimize = 'optimal')
name += 2.0*np.einsum('lmde, lmde -> ', T_OoVv, V_OOVV, optimize = 'optimal')



$\bullet$ The output is more useful if we define some options

In [19]:
mytensors = {
    'T_OoVv' : 'T2',
    'T_OV' : 'T1',
    'T_OoOVvV': 'T3',
    'V_OOOO' : 'Voooo',
    'V_OOOV' : 'Vooov',
    'V_OOVV' : 'Voovv',
    'V_OVOV' : 'Vovov',
    'V_OVVV' : 'Vovvv',
    'V_VVVV' : 'Vvvvv',
    'f_OO' : 'fock_OO',
    'f_VV' : 'fock_VV',
    'f_OV' : 'fock_OV'
    
}

pE = process_eq(eq=E, name = 'CC_energy', tensor_labels = mytensors)
pE.write_einsums_out(print_out=True)

CC_energy += 2.0*np.einsum('ld, ld -> ', fock_OV, T1, optimize = 'optimal')
CC_energy -= np.einsum('md, le, lmde -> ', T1, T1, Voovv, optimize = 'optimal')
CC_energy += 2.0*np.einsum('md, le, mlde -> ', T1, T1, Voovv, optimize = 'optimal')
CC_energy -= np.einsum('mlde, lmde -> ', T2, Voovv, optimize = 'optimal')
CC_energy += 2.0*np.einsum('lmde, lmde -> ', T2, Voovv, optimize = 'optimal')



## Processing $T_1$ amplitude equation

In [20]:
T1 = eqfromlatex(latex(eqT1), index_key)
platex(T1)

<IPython.core.display.Math object>

$\bullet$ Simplify

In [21]:
T1 = T1.simplify()
T1 = T1.adapt_space()
platex(T1)

<IPython.core.display.Math object>

In [22]:
pT1 = process_eq(T1,i,a,name = 'newT1', tensor_labels = mytensors)
pT1.write_einsums_out(print_out=True)

newT1 -= np.einsum('ld, id, la -> ia', fock_OV, T1, T1, optimize = 'optimal')
newT1 += 2.0*np.einsum('ld, ilad -> ia', fock_OV, T2, optimize = 'optimal')
newT1 -= np.einsum('ld, liad -> ia', fock_OV, T2, optimize = 'optimal')
newT1 -= np.einsum('il, la -> ia', fock_OO, T1, optimize = 'optimal')
newT1 += np.einsum('da, id -> ia', fock_VV, T1, optimize = 'optimal')
newT1 += fock_OV
newT1 += -2.0*np.einsum('ld, ie, ma, lmde -> ia', T1, T1, T1, Voovv, optimize = 'optimal')
newT1 += np.einsum('ld, ie, ma, mlde -> ia', T1, T1, T1, Voovv, optimize = 'optimal')
newT1 += 2.0*np.einsum('ld, ie, lade -> ia', T1, T1, Vovvv, optimize = 'optimal')
newT1 -= np.einsum('ld, ie, laed -> ia', T1, T1, Vovvv, optimize = 'optimal')
newT1 += -2.0*np.einsum('ld, ma, mlid -> ia', T1, T1, Vooov, optimize = 'optimal')
newT1 += np.einsum('ld, ma, lmid -> ia', T1, T1, Vooov, optimize = 'optimal')
newT1 += 4.0*np.einsum('ld, imae, lmde -> ia', T1, T2, Voovv, optimize = 'optimal')
newT1 += -2.0*np.einsum('ld, imae, 

## Processing $T_2$ amplitude equation

In [23]:
T2 = eqfromlatex(latex(eqT2), index_key)
platex(T2)

<IPython.core.display.Math object>

$\bullet$ Simplify

In [24]:
T2 = T2.simplify()
T2 = T2.adapt_space()
platex(T2)

<IPython.core.display.Math object>

In [25]:
pT2 = process_eq(T2,i,j,a,b, name = 'newT2', tensor_labels = mytensors)
pT2.write_einsums_out(print_out=True)

newT2 -= np.einsum('ld, id, ljab -> ijab', fock_OV, T1, T2, optimize = 'optimal')
newT2 -= np.einsum('ld, jd, ilab -> ijab', fock_OV, T1, T2, optimize = 'optimal')
newT2 -= np.einsum('ld, la, ijdb -> ijab', fock_OV, T1, T2, optimize = 'optimal')
newT2 -= np.einsum('ld, lb, ijad -> ijab', fock_OV, T1, T2, optimize = 'optimal')
newT2 += np.einsum('ld, jilbad -> ijab', fock_OV, T3, optimize = 'optimal')
newT2 += np.einsum('ld, ijlabd -> ijab', fock_OV, T3, optimize = 'optimal')
newT2 -= np.einsum('il, ljab -> ijab', fock_OO, T2, optimize = 'optimal')
newT2 -= np.einsum('jl, ilab -> ijab', fock_OO, T2, optimize = 'optimal')
newT2 += np.einsum('da, ijdb -> ijab', fock_VV, T2, optimize = 'optimal')
newT2 += np.einsum('db, ijad -> ijab', fock_VV, T2, optimize = 'optimal')
newT2 += -2.0*np.einsum('ld, ie, mjab, lmde -> ijab', T1, T1, T2, Voovv, optimize = 'optimal')
newT2 += np.einsum('ld, ie, mjab, mlde -> ijab', T1, T1, T2, Voovv, optimize = 'optimal')
newT2 += -2.0*np.einsum('ld, je, imab, 

## Processing $T_3$ amplitude equation

In [26]:
T3 = eqfromlatex(latex(eqT3), index_key)
platex(T3)

<IPython.core.display.Math object>

$\bullet$ Simplify

In [27]:
T3 = T3.simplify()
T3 = T3.adapt_space()
platex(T3)

<IPython.core.display.Math object>

In [29]:
T3.sort()

In [30]:
pT3 = process_eq(T3,i,j,k,a,b,c, name = 'newT3', tensor_labels = mytensors)
pT3.write_einsums_out(print_out=True)

newT3 += np.einsum('dc, ijkabd -> ijkabc', fock_VV, T3, optimize = 'optimal')
newT3 += np.einsum('db, ijkadc -> ijkabc', fock_VV, T3, optimize = 'optimal')
newT3 -= np.einsum('da, ijkcbd -> ijkabc', fock_VV, T3, optimize = 'optimal')
newT3 -= np.einsum('kl, ijlabc -> ijkabc', fock_OO, T3, optimize = 'optimal')
newT3 -= np.einsum('jl, ilkabc -> ijkabc', fock_OO, T3, optimize = 'optimal')
newT3 += np.einsum('il, kjlabc -> ijkabc', fock_OO, T3, optimize = 'optimal')
newT3 += np.einsum('ld, id, kjlabc -> ijkabc', fock_OV, T1, T3, optimize = 'optimal')
newT3 += np.einsum('kicd, jdba -> ijkabc', T2, Vovvv, optimize = 'optimal')
newT3 -= np.einsum('ijcd, kdab -> ijkabc', T2, Vovvv, optimize = 'optimal')
newT3 -= np.einsum('klcb, jila -> ijkabc', T2, Vooov, optimize = 'optimal')
newT3 += np.einsum('ljcb, ikla -> ijkabc', T2, Vooov, optimize = 'optimal')
newT3 -= np.einsum('ljcb, kila -> ijkabc', T2, Vooov, optimize = 'optimal')
newT3 += np.einsum('ilcb, jkla -> ijkabc', T2, Vooov, optimize = '

## The equations above can be coded into a CC program. See the RCCSD.py file for a complete code